## FABlib API References Examples

- [fablib.show_config](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.show_config)
- [fablib.list_sites](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_sites)
- [fablib.list_hosts](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_hosts)
- [fablib.new_slice](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.new_slice)
- [slice.add_node](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.add_node)
- [slice.submit](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.submit)
- [slice.get_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.get_nodes)
- [slice.list_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.list_nodesß)
- [slice.show](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.show)
- [node.execute](https://fabric-fablib.readthedocs.io/en/latest/node.html#fabrictestbed_extensions.fablib.node.Node.execute)
- [slice.delete](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.delete) 

In [1]:
import datetime

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,49f65ad7-d8a2-4ab9-8ca0-ba777a2e0ea2
Bastion Host,bastion.fabric-testbed.net
Bastion Username,jakejongejans_0000313927
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


In [3]:
slice_name = 'DYNAMOS-on-FABRIC'
image = "default_ubuntu_24"

node_configurations = [
    {
        "name": "control",
        "cores": 2,
        "ram": 8,
        "disk": 100,
        "site": "AMST",
        "host": "amst-w3.fabric-testbed.net",
    },
    {
        "name": "dynamos",
        "cores": 4,
        "ram": 16,
        "disk": 100,
        "site": "AMST",
        "host": "amst-w3.fabric-testbed.net",
    },
    {
        "name": "uva",
        "cores": 4,
        "ram": 16,
        "disk": 100,
        "site": "AMST",
        "host": "amst-w3.fabric-testbed.net",
    },
    {
        "name": "vu",
        "cores": 4,
        "ram": 16,
        "disk": 100,
        "site": "LOSA",
        "host": "losa-w3.fabric-testbed.net",
    },
    {
        "name": "surf",
        "cores": 8,
        "ram": 32,
        "disk": 100,
        "site": "TOKY",
        "host": "toky-w3.fabric-testbed.net",
    }
]

sites = list(set([configuration["site"] for configuration in node_configurations]))

def create_node(slice, configuration):
    return slice.add_node(name=configuration["name"], 
                          site=configuration["site"], 
                          host=configuration["host"], 
                          cores=configuration["cores"], 
                          ram=configuration["ram"], 
                          disk=configuration["disk"], 
                          validate=True, 
                          raise_exception=True, 
                          image=image)
    

In [24]:
# Create a slice
slice = fablib.new_slice(name=slice_name)

# Add Nodes with the specific variables
# Also validate the node can be created and raise an exception in case of failure
print('Adding nodes...')
nodes = [create_node(slice, configuration) for configuration in node_configurations]
nodes_per_site = [
    (site, [node for node in nodes if node.get_site() == site])
    for site in sites
]

print('Adding network interfaces...')
interfaces_per_site = [
    (site, [node.add_component(model='NIC_Basic', name='NIC').get_interfaces()[0] for node in nodes])
    for (site, nodes) in nodes_per_site
]

print('Adding network...')
networks = [
    slice.add_l3network(name=f'Network-{site}', interfaces=interfaces, type="IPv4")
    for (site, interfaces) in interfaces_per_site
]

print(networks, [n.get_gateway() for n in networks], [n.get_subnet() for n in networks])

# Calculate the lease end time for 2 weeks from now with timezone information
lease_end_time = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(weeks=2)

# Submit the slice, using an end date 2 weeks from now (the current maximum lease time) 
# to make sure that the slice can be used for a longer period of time. Progress shows an indicator of the current progression.
# Wait until the state is finished and use an interval (it may take some time before the slice and nodes are created)
print('Creating slice...')
slice.submit(wait=True, wait_timeout=3600, wait_interval=20, progress=True, wait_jupyter='text', lease_end_time=lease_end_time);


Retry: 18, Time: 450 sec


ID,bac990ec-f502-47fe-a729-67fae05b47c2
Name,DYNAMOS-on-FABRIC
Lease Expiration (UTC),2025-05-02 13:38:03 +0000
Lease Start (UTC),2025-04-18 13:38:03 +0000
Project ID,49f65ad7-d8a2-4ab9-8ca0-ba777a2e0ea2
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
a802d6e7-1f40-4321-a7ae-9c74aa335a7f,control,2,8,100,default_ubuntu_24,qcow2,amst-w3.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:feee:b312,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:feee:b312,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
ba0c222b-79ab-485a-a493-262457ed59e6,dynamos,4,16,100,default_ubuntu_24,qcow2,amst-w3.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:feb6:5aba,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:feb6:5aba,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
cea2a325-d3bf-4cf9-960c-7098010a48cd,surf,8,32,100,default_ubuntu_24,qcow2,toky-w3.fabric-testbed.net,TOKY,ubuntu,133.69.160.242,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.242,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
487f8fcd-e629-4790-9239-272c6df56e6e,uva,4,16,100,default_ubuntu_24,qcow2,amst-w3.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe25:ac7b,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe25:ac7b,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
952ebd20-693f-48d5-8a0e-942db3988c09,vu,4,16,100,default_ubuntu_24,qcow2,losa-w3.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:febb:9ce3,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:febb:9ce3,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
619ca71f-c68e-4af0-b3b5-58cb7f009100,Network-AMST,L3,FABNetv4,AMST,10.145.3.0/24,10.145.3.1,Active,
fea3d78f-fa0f-486c-a2a4-07ce36d3d2c7,Network-LOSA,L3,FABNetv4,LOSA,10.137.8.0/24,10.137.8.1,Active,
944a9769-f30e-4ae8-8932-6e065c0c0292,Network-TOKY,L3,FABNetv4,TOKY,10.146.8.0/24,10.146.8.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
control-NIC-p1,p1,control,Network-AMST,100,config,,0A:40:F0:AF:E6:92,enp7s0,enp7s0,fe80::840:f0ff:feaf:e692,4,HundredGigE0/0/0/9
dynamos-NIC-p1,p1,dynamos,Network-AMST,100,config,,0A:72:73:1C:13:2A,enp7s0,enp7s0,fe80::872:73ff:fe1c:132a,4,HundredGigE0/0/0/9
uva-NIC-p1,p1,uva,Network-AMST,100,config,,0A:B1:A5:3F:0F:02,enp7s0,enp7s0,fe80::8b1:a5ff:fe3f:f02,4,HundredGigE0/0/0/9
vu-NIC-p1,p1,vu,Network-LOSA,100,config,,06:08:75:E3:98:05,enp6s0,enp6s0,fe80::408:75ff:fee3:9805,4,HundredGigE0/0/0/9
surf-NIC-p1,p1,surf,Network-TOKY,100,config,,0A:A8:67:3B:04:90,enp7s0,enp7s0,fe80::8a8:67ff:fe3b:490,4,HundredGigE0/0/0/9



Time to print interfaces 471 seconds


In [4]:
slice = fablib.get_slice(name="DYNAMOS-on-FABRIC");
nodes = slice.get_nodes();
nodes_and_network_per_site = [
    (site, [node for node in nodes if node.get_site() == site], slice.get_network(name=f"Network-{site}"))
    for site in sites
]

nodes_network_ips_per_site = [
    (site, nodes, network, network.get_available_ips(len(nodes)))
    for (site, nodes, network) in nodes_and_network_per_site
]

In [ ]:
slice.list_nodes();

In [ ]:
for node in slice.get_nodes():
    stdout, stderr = node.execute('echo Hello, FABRIC from node `hostname -s`')

In [42]:
def assign_ip(site, network, available_ips, node):
    interface = node.get_interface(network_name=f"Network-{site}")
    address = available_ips.pop(0)
    network_gateway = network.get_gateway()
    network_subnet = network.get_subnet()

    network.allocate_ip(address)
    interface.ip_addr_add(addr=address, subnet=network_subnet)
    node.ip_route_add(subnet=network_subnet, gateway=network_gateway)

    # For the multisite IPv4 connection
    for network in networks:
        node.ip_route_add(subnet=network.get_subnet(), gateway=network_gateway)

    return address

ips = [assign_ip(site, network, ips, node) for (site, nodes, network, ips) in nodes_network_ips_per_site for node in nodes];

print(ips);


[IPv4Address('10.146.8.2'), IPv4Address('10.145.3.2'), IPv4Address('10.145.3.3'), IPv4Address('10.145.3.4'), IPv4Address('10.137.8.2')]


In [5]:
for node in nodes:
    ssh_command = node.get_ssh_command().replace(
        "-i /home/fabric/work/fabric_config/slice_key", "-i ~/.ssh/keys/FABRIC-slice_key"
    ).replace(
        "-F /home/fabric/work/fabric_config/ssh_config ", ""
    )
    
    print(ssh_command);

ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:610:2d0:fabc:f816:3eff:feee:b312
ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:610:2d0:fabc:f816:3eff:feb6:5aba
ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:610:2d0:fabc:f816:3eff:fe25:ac7b
ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:400:a100:3070:f816:3eff:febb:9ce3
ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@133.69.160.242


In [7]:
for node in nodes:
    print(node.get_name(), node.get_interface(network_name=f"Network-{node.get_site()}"))

control ---------------  ------------------
Name             control-NIC-p1
Network          Network-AMST
Bandwidth        100
Mode             config
VLAN
MAC              0A:40:F0:AF:E6:92
Physical Device  enp7s0
Device           enp7s0
Address          10.145.3.2
Numa Node        4
Switch Port      HundredGigE0/0/0/9
---------------  ------------------
dynamos ---------------  ------------------
Name             dynamos-NIC-p1
Network          Network-AMST
Bandwidth        100
Mode             config
VLAN
MAC              0A:72:73:1C:13:2A
Physical Device  enp7s0
Device           enp7s0
Address          10.145.3.3
Numa Node        4
Switch Port      HundredGigE0/0/0/9
---------------  ------------------
uva ---------------  ------------------
Name             uva-NIC-p1
Network          Network-AMST
Bandwidth        100
Mode             config
VLAN
MAC              0A:B1:A5:3F:0F:02
Physical Device  enp7s0
Device           enp7s0
Address          10.145.3.4
Numa Node        4
Switch